## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
names = application_df[['EIN', 'NAME']]
noname_df = application_df.drop(['EIN', 'NAME'], axis=1)

noname_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
noname_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

`APPLICATION_TYPE` and `CLASSIFICATION`, both have more than 10 unique values so they should be examined more closely to determine which values appear enough times to be individually relevant and which are rare enough to be binned together.
<br>
While `ASK_AMT` also has many unique values, it is a numeric column and thus the full range of values is relevant.

In [6]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = noname_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# set cutoff to 200, create dictionary from value_counts result, and initialize list for app types to replace
cutoff = 200
app_type_count_dict = dict(app_type_counts)
application_types_to_replace = []

# iterate through items in dictionary, add app types with value < cutoff to list
for key, value in app_type_count_dict.items():
    if value < cutoff:
        application_types_to_replace.append(key)

# create copy of df for reduced app types
red_app_type_df = noname_df

# Replace in dataframe
for app in application_types_to_replace:
    red_app_type_df['APPLICATION_TYPE'] = red_app_type_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
red_app_type_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_counts = red_app_type_df['CLASSIFICATION'].value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
class_counts.loc[class_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# set cutoff to 1000, create dictionary from value_counts result, and initialize list for classifications to replace
cutoff = 1000
class_count_dict = dict(class_counts)
classifications_to_replace = []

# iterate through items in dictionary, add classifications with value < cutoff to list
for key, value in class_count_dict.items():
    if value < cutoff:
        classifications_to_replace.append(key)

# create copy of dataframe for reduced classifications
red_class_df = red_app_type_df

# Replace in dataframe
for cls in classifications_to_replace:
    red_class_df['CLASSIFICATION'] = red_class_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
red_class_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(red_class_df)
dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [12]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop('IS_SUCCESSFUL', axis=1)
y = dummies_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimization 1

Added one more hidden layer to the model.

## Compile, Train and Evaluate the Model 

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

#  build the model
nn_model_op1 = tf.keras.models.Sequential()

# First hidden layer
nn_model_op1.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# New hidden layer 2
nn_model_op1.add(tf.keras.layers.Dense(units=60, activation="relu"))

# third hidden layer
nn_model_op1.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_op1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_op1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 60)                4860      
                                                                 
 dense_2 (Dense)             (None, 30)                1830      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 10241 (40.00 KB)
Trainable params: 10241 (40.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn_model_op1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model_op1 = nn_model_op1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 830us/step - loss: 0.5689 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 1s 882us/step - loss: 0.5536 - accuracy: 0.7295
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7309
Epoch 4/100
804/804 [==============================] - 1s 885us/step - loss: 0.5506 - accuracy: 0.7302
Epoch 5/100
804/804 [==============================] - 1s 805us/step - loss: 0.5494 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 1s 857us/step - loss: 0.5486 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 1s 899us/step - loss: 0.5480 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 1s 856us/step - loss: 0.5476 - accuracy: 0.7331
Epoch 9/100
804/804 [==============================] - 1s 917us/step - loss: 0.5467 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 1s 913us/step - los

In [17]:
# Evaluate the model using the test data
model_loss_op1, model_accuracy_op1 = nn_model_op1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_op1}, Accuracy: {model_accuracy_op1}")

268/268 - 0s - loss: 0.5729 - accuracy: 0.7275 - 275ms/epoch - 1ms/step
Loss: 0.572936475276947, Accuracy: 0.7274635434150696


In [32]:
# Export our model to HDF5 file
nn_model_op1.save('Output/AlphabetSoupCharity_Optimization1.h5')

The first optimization attempt failed. Accuracy indicated a slight drop.

## Optimization 2

Reduced number of layers and added more neurons per layer.

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

#  build the model
nn_model_op2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_op2.add(tf.keras.layers.Dense(units=160, activation="relu", input_dim=43))

# Second hidden layer
nn_model_op2.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Output layer
nn_model_op2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_op2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 160)               7040      
                                                                 
 dense_5 (Dense)             (None, 60)                9660      
                                                                 
 dense_6 (Dense)             (None, 1)                 61        
                                                                 
Total params: 16761 (65.47 KB)
Trainable params: 16761 (65.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn_model_op2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model_op2 = nn_model_op2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


804/804 [==============================] - 1s 935us/step - loss: 0.5683 - accuracy: 0.7235
Epoch 2/100
804/804 [==============================] - 1s 912us/step - loss: 0.5569 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 1s 895us/step - loss: 0.5526 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 1s 909us/step - loss: 0.5495 - accuracy: 0.7311
Epoch 6/100
804/804 [==============================] - 1s 880us/step - loss: 0.5495 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 1s 927us/step - loss: 0.5478 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 1s 883us/step - loss: 0.5462 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 1s 877us/step - loss: 0.5461 - ac

In [22]:
# Evaluate the model using the test data
model_loss_op2, model_accuracy_op2 = nn_model_op2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_op2}, Accuracy: {model_accuracy_op2}")

268/268 - 0s - loss: 0.5645 - accuracy: 0.7287 - 245ms/epoch - 913us/step
Loss: 0.5645090937614441, Accuracy: 0.7287463545799255


In [23]:
# Export our model to HDF5 file
nn_model_op2.save('Output/AlphabetSoupCharity_Optimization2.h5')

The second attempt at optimazation failed. While neurons increased, the model accuracy decreased.

## Optimization 3

Used the original neural net model and dropped columns deemed less important.
<br>
 `STATUS` and `SPECIAL_CONSIDERATIONS` columns were dropped.

## Data Reprocessing 

In [24]:
# call the processed data just prior to categorical conversion and drop selected columns
op3_df = red_class_df.drop(['STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1)

# convert categorical data to numeric
op3_dummies_df = pd.get_dummies(op3_df)
op3_dummies_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [25]:
# Split our preprocessed data into our features and target arrays
X_op3 = op3_dummies_df.drop('IS_SUCCESSFUL', axis=1)
y_op3 = op3_dummies_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_op3_train, X_op3_test, y_op3_train, y_op3_test = train_test_split(X_op3, y_op3, random_state=1)

In [26]:
# Create a StandardScaler instances
scaler_op3 = StandardScaler()

# Fit the StandardScaler
X_scaler_op3 = scaler_op3.fit(X_op3_train)

# Scale the data
X_op3_train_scaled = X_scaler_op3.transform(X_op3_train)
X_op3_test_scaled = X_scaler_op3.transform(X_op3_test)

## Compile, Train and Evaluate the Model

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

#  build the model
nn_model_op3 = tf.keras.models.Sequential()

# First hidden layer
nn_model_op3.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=40))

# Second hidden layer
nn_model_op3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_op3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_op3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3280      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5741 (22.43 KB)
Trainable params: 5741 (22.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn_model_op3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model_op3 = nn_model_op3.fit(X_op3_train_scaled, y_op3_train, epochs=100)

Epoch 1/100


804/804 [==============================] - 1s 831us/step - loss: 0.5692 - accuracy: 0.7218
Epoch 2/100
804/804 [==============================] - 1s 871us/step - loss: 0.5556 - accuracy: 0.7285
Epoch 3/100
804/804 [==============================] - 1s 852us/step - loss: 0.5526 - accuracy: 0.7285
Epoch 4/100
804/804 [==============================] - 1s 922us/step - loss: 0.5505 - accuracy: 0.7295
Epoch 5/100
804/804 [==============================] - 1s 808us/step - loss: 0.5493 - accuracy: 0.7305
Epoch 6/100
804/804 [==============================] - 1s 856us/step - loss: 0.5488 - accuracy: 0.7322
Epoch 7/100
804/804 [==============================] - 1s 802us/step - loss: 0.5477 - accuracy: 0.7317
Epoch 8/100
804/804 [==============================] - 1s 797us/step - loss: 0.5473 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 1s 810us/step - loss: 0.5462 - accuracy: 0.7331
Epoch 10/100
804/804 [==============================] - 1s 811us/step - loss: 0.5461 

In [30]:
# Evaluate the model using the test data
model_loss_op3, model_accuracy_op3 = nn_model_op3.evaluate(X_op3_test_scaled,y_op3_test,verbose=2)
print(f"Loss: {model_loss_op3}, Accuracy: {model_accuracy_op3}")

268/268 - 0s - loss: 0.5586 - accuracy: 0.7287 - 241ms/epoch - 901us/step
Loss: 0.5585694313049316, Accuracy: 0.7287463545799255


This optimization attempt equally failed. Dropped columns had little to no  impact on the accuracy of the model, with only a trivial decrease.

In [31]:
# Export our model to HDF5 file
nn_model_op3.save('Output/AlphabetSoupCharity_Optimization3.h5')

c:\Users\cuteo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
